In [ ]:
import base64
import os

# Função para converter PDF em Base64
def convert_pdf_to_base64(pdf_path):
    with open(pdf_path, "rb") as pdf_file:
        # Lê o arquivo PDF como binário e converte para base64
        encoded_pdf = base64.b64encode(pdf_file.read()).decode('utf-8')
    return encoded_pdf

# Função para salvar o base64 em arquivo txt
def save_base64_to_txt(base64_content, output_txt_path):
    with open(output_txt_path, 'w', encoding='utf-8') as txt_file:
        txt_file.write(base64_content)

# Função principal para processar todos os PDFs no diretório
def process_pdf_files(input_directory, output_directory):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    for filename in os.listdir(input_directory):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(input_directory, filename)
            base64_content = convert_pdf_to_base64(pdf_path)
            
            # Define o caminho do arquivo txt de saída
            output_txt_path = os.path.join(output_directory, f"{os.path.splitext(filename)[0]}.txt")
            
            # Salva o conteúdo Base64 em um arquivo txt
            save_base64_to_txt(base64_content, output_txt_path)
            print(f"Arquivo convertido e salvo: {output_txt_path}")

if __name__ == "__main__":
    # Diretórios de entrada e saída (substitua pelos seus caminhos)
    input_pdf_directory = "/Users/xyz/abc/xyz"
    output_txt_directory = "/Users/xyz/abc/xyz"

    # Processa os arquivos PDF
    process_pdf_files(input_pdf_directory, output_txt_directory)


In [ ]:
# import modules
from elasticsearch import Elasticsearch
from getpass import getpass
from urllib.request import urlopen
import json
from time import sleep

In [ ]:
from pathlib import Path
from eland.ml.pytorch import PyTorchModel
from eland.ml.pytorch.transformers import TransformerModel
from elasticsearch import Elasticsearch
from elasticsearch.client import MlClient

import getpass

In [ ]:
es_cloud_id = getpass.getpass('Enter Elastic Cloud ID:  ')
es_user = getpass.getpass('Enter cluster username:  ') 
es_pass = getpass.getpass('Enter cluster password:  ') 

#es_api_id = getpass.getpass('Enter cluster API key ID:  ') 
#es_api_key = getpass.getpass('Enter cluster API key:  ')

In [ ]:
#es = Elasticsearch(cloud_id=es_cloud_id, 
#                   api_key=(es_api_id, es_api_key)
#                   )
es = Elasticsearch(cloud_id=es_cloud_id, 
                   basic_auth=(es_user, es_pass)
                   )
es.info() # should return cluster info
     

In [ ]:
from elasticsearch import Elasticsearch
import os

# Função para ler um arquivo .txt e armazenar o conteúdo
def read_txt_file(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            file_content = file.read()
        return file_content
    else:
        raise FileNotFoundError(f"Arquivo {file_path} não encontrado.")

# Função para inserir um arquivo no Elasticsearch
def insert_file_into_elasticsearch(file_path, es_client, file_id):
    try:
        file_content = read_txt_file(file_path)
        # Inserindo o arquivo no Elasticsearch com o ID e pipeline especificados
        response = es_client.index(
            index="search-index-name-pt",
            id=file_id,  # ID exclusivo para cada arquivo
            body={
                "filename": os.path.basename(file_path),
                "data": file_content  # Inserindo o conteúdo no campo 'data'
            },
            pipeline="text-embeddings-frompdf-pt"  # Usando a pipeline de ingestão
        )
        print(f"Arquivo {file_path} inserido com sucesso com ID {file_id}: {response}")
    except Exception as e:
        print(f"Erro ao inserir {file_path}: {e}")

if __name__ == "__main__":
    # Caminho do diretório onde estão os arquivos .txt
    txt_directory = "/Users/xyz/abc/xyz"  # Substitua pelo caminho correto

    # Iterando sobre os arquivos do diretório e inserindo um por um
    file_id = 1  # Começa com ID 1
    for filename in os.listdir(txt_directory):
        if filename.endswith(".txt"):
            file_path = os.path.join(txt_directory, filename)
            # Inserindo cada arquivo com um ID exclusivo
            insert_file_into_elasticsearch(file_path, es, file_id)
            file_id += 1  # Incrementa o ID para o próximo arquivo
